In [1]:
import pandas as pd
import requests
import httpx
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
from random import choice
import sqlite3
pd.set_option('display.max_columns', None)

In [2]:
conn = sqlite3.connect("nbaDatasets/nba_database.db")
existing_sched = pd.read_sql_query("SELECT * FROM Schedule", conn)
# existing_sched = pd.read_sql_query("SELECT * FROM playerBoxScores", conn)

conn.close()
existing_sched = pd.DataFrame(existing_sched)

# view_df['team_displayName'].unique()

In [3]:
existing_sched.sort_values(by=['event_season', 'event_startDate', 'event_startTime']).tail(20)


,event_id,event_season,event_neutralSite,event_boxScoreAvailable,venue_name,home_id,home_score,away_id,away_score,status_completed,status_description,event_startDate,event_startTime
39206,401705107,2025,0,1,United Center,4,119,23,124,1,Final,2025-01-12,14:30:00
39207,401705108,2025,0,1,TD Garden,2,120,3,119,1,Final,2025-01-12,17:00:00
39208,401705109,2025,0,1,Rocket Mortgage FieldHouse,5,93,11,108,1,Final,2025-01-12,17:00:00
39209,401705110,2025,0,1,Kia Center,19,104,20,99,1,Final,2025-01-12,17:00:00
39210,401705111,2025,0,1,Capital One Arena,27,95,25,136,1,Final,2025-01-12,17:00:00
39211,401705112,2025,0,1,Delta Center,26,112,17,111,1,Final,2025-01-12,19:00:00
39212,401705113,2025,0,1,Footprint Center,21,120,30,113,1,Final,2025-01-12,20:00:00
39213,401705114,2025,0,1,Capital One Arena,27,106,16,120,1,Final,2025-01-13,18:00:00
39214,401705115,2025,0,1,Madison Square Garden,18,119,8,124,1,Final,2025-01-13,18:30:00
39215,401705116,2025,0,1,Scotiabank Arena,28,104,9,101,1,Final,2025-01-13,18:30:00


In [4]:
sport_name = 'basketball'
league_name = 'nba'
team_urls = httpx.get(f'https://sports.core.api.espn.com/v2/sports/{sport_name}/leagues/{league_name}/teams?limit=1000').json().get('items')
team_urls = [team_url['$ref'] for team_url in team_urls]
team_url = team_urls[0]

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=10))
def get_team_data(team_url):
    try:
        team_response = httpx.get(team_url)
        team_response.raise_for_status()
        team_data = team_response.json()

        team_dict = {
            'team.id' : team_data.get('id'), # Primary Key
            'team.guid' : team_data.get('guid'),
            'team.uid' : team_data.get('uid'),
            'team.slug' : team_data.get('slug'),
            'team.location' : team_data.get('location'),
            'team.name' : team_data.get('name'),
            'team.abbreviation' : team_data.get('abbreviation'),
            'team.displayName' : team_data.get('displayName'),
            'team.color' : team_data.get('color'),
            'team.alternateColor' : team_data.get('alternateColor'),
            'team.venue.id' : team_data.get('venue',{}).get('id'),
            'team.logo' : f"https://a.espncdn.com/i/teamlogos/nba/500/{team_data.get('abbreviation')}.png",
            'conference.id' : team_data.get('groups',{}).get('$ref').split('?')[0].split('/')[-1]
        }
        return team_dict
    except Exception as e:
        print(f"Error fetching data for URL {team_url}: {e}")

def get_all_teams_data(team_urls):
    teams_df = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        # Prepare future tasks for each combination of team ID and date range
        future_to_sched = {
            executor.submit(get_team_data, team_url): (team_urls)
            for team_url in team_urls
        }
        # As each future completes, append its result to fullSched
        for future in as_completed(future_to_sched):
            sched = future.result()
            teams_df.append(sched)
    return teams_df

teams_df = pd.DataFrame(get_all_teams_data(team_urls))
teams_df['team.id'] = teams_df['team.id'].astype(int)
teams_df = teams_df.sort_values(by='team.id')

conn = sqlite3.connect("nbaDatasets/nba_database.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Teams (
               team_id INTEGER PRIMARY KEY,
               team_guid TEXT,
               team_uid TEXT,
               team_slug TEXT,
               team_location TEXT,
               team_abbreviation TEXT,
               team_displayName TEXT,
               team_color TEXT,
               team_alternateColor TEXT,
               team_venue_id INTEGER,
               team_logo TEXT,
               conference_id INTEGER
               )
               """)

for index, row in teams_df.iterrows():
    cursor.execute("""
    INSERT OR REPLACE INTO Teams (
                   team_id, 
                   team_guid, 
                   team_uid, 
                   team_slug, 
                   team_location, 
                   team_abbreviation, 
                   team_displayName, 
                   team_color, 
                   team_alternateColor,
                   team_venue_id,
                   team_logo,
                   conference_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                   """, (
                       row['team.id'],
                       row['team.guid'],
                       row['team.uid'],
                       row['team.slug'],
                       row['team.location'],
                       row['team.abbreviation'],
                       row['team.displayName'],
                       row['team.color'],
                       row['team.alternateColor'],
                       row['team.venue.id'],
                       row['team.logo'],
                       row['conference.id']

            ))
conn.commit()
conn.close()

In [5]:
team_ids = list(teams_df['team.id'].unique())



def get_sched_for_team(season, team_id):

    sched_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams/{team_id}/schedule?season={season}"


    try:
        sched_response = httpx.get(sched_url)
        sched_response.raise_for_status()
        sched_data = sched_response.json()
        event_data = sched_data.get('events',[])

        sched_df = []
        for event in event_data:
            competition_data = event.get('competitions',[{}])[0]
            event_dict = {
                'event_id' : competition_data.get('id'),
                'event_date' : competition_data.get('date'),
                'event_season' : season,
                'event_neutralSite' : competition_data.get('neutralSite'),
                'event_boxScoreAvailable' : competition_data.get('boxscoreAvailable'),
                'venue_name' : competition_data.get('venue',{}).get('fullName')
            }
            for competitor in competition_data.get('competitors',[]):
                homeAway = competitor.get('homeAway')
                event_dict[f"{homeAway}_id"] = competitor.get('id')
                event_dict[f"{homeAway}_score"] = competitor.get('score',{}).get('value')
            
            event_dict['status_completed'] = competition_data.get('status',{}).get('type',{}).get('completed')
            event_dict['status_description'] = competition_data.get('status',{}).get('type',{}).get('description')

            sched_df.append(event_dict)
        return sched_df
    except Exception as e:
        print(f"Error fetching data for URL {sched_url}: {e}")

def get_all_teams_sched(team_ids):
    sched_df = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        # Prepare future tasks for each combination of team ID and date range
        future_to_sched = {
            executor.submit(get_sched_for_team, season, team_id): (team_ids)
            for team_id in team_ids
            for season in range(2025,2026)
        }
        # As each future completes, append its result to fullSched
        for future in as_completed(future_to_sched):
            sched = future.result()
            sched_df.extend(sched)
    return sched_df

sched_df = get_all_teams_sched(team_ids)
sched_df = pd.DataFrame(sched_df)
sched_df = sched_df.drop_duplicates(keep='first')
# Assuming your dataframe is named 'df'
sched_df['temp_datetime'] = pd.to_datetime(sched_df['event_date']) - pd.Timedelta(hours=6)

# Extract date and time into separate columns
sched_df['event_startDate'] = sched_df['temp_datetime'].dt.date
sched_df['event_startTime'] = sched_df['temp_datetime'].dt.time

# Drop the original and temporary columns
sched_df = sched_df.drop(['event_date', 'temp_datetime'], axis=1)
sched_df = sched_df.sort_values(by=['event_startDate','event_startTime'])
sched_df = sched_df[sched_df['status_completed']]


In [6]:
existing_sched_ids = list(existing_sched['event_id'].astype(str).unique())
sched_df_ids = list(sched_df['event_id'].astype(str).unique())

new_ids = [x for x in sched_df_ids if x not in existing_sched_ids]
new_ids

['401705127',
 '401705128',
 '401705129',
 '401705132',
 '401705130',
 '401705133',
 '401705135',
 '401705134',
 '401705131',
 '401705136',
 '401705137']

In [7]:
def grab_event_data(event_id):

    event_url = f"https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/summary?event={event_id}"



    try:
        event_response = httpx.get(event_url, timeout=10)
        event_response.raise_for_status()

        event_data = event_response.json()

        team_boxScores = event_data.get('boxscore',{}).get('teams',[])
        player_boxScores = event_data.get('boxscore',{}).get('players',[])

        
        teamBoxScores_df = []
        for team in team_boxScores:
            team_dict = team.get('team',{})
            team_BoxScore_dict = {
                'event_id' : event_id,
                'team_id' : team_dict.get('id')
            }
            for statistic in team.get('statistics',[]):
                if '-' not in statistic.get('name'):
                    team_BoxScore_dict[f"{statistic.get('name')}"] = statistic.get('displayValue')
                else:

                    key1 = statistic.get('name').split('-')[0]
                    key2 = statistic.get('name').split('-')[1]
                    value1 = statistic.get('displayValue').split('-')[0]
                    value2 = statistic.get('displayValue').split('-')[1]

                    team_BoxScore_dict[f"{key1}"] = value1
                    team_BoxScore_dict[f"{key2}"] = value2




            teamBoxScores_df.append(team_BoxScore_dict)
        
        playerBoxScores_df = []
        for team in player_boxScores:
            team_dict = team.get('team',{})
            player_stats = team.get('statistics',[{}])[0]
            player_dict_keys = player_stats.get('keys')

            for athlete in player_stats.get('athletes',[]):

                athlete_boxScore_dict = {
                    'event_id' : event_id,
                    'team_id' : team_dict.get('id'),
                    'team_displayName' : team_dict.get('displayName'),
                    'athlete_active' : athlete.get('active'),
                    'athlete_starter' : athlete.get('starter'),
                    'athlete_didNotPlay' : athlete.get('didNotPlay'),
                    'athlete_reason' : athlete.get('reason'),
                    'athlete_ejected' : athlete.get('ejected'),
                    'athlete_id' : athlete.get('athlete',{}).get('id'),
                    'athlete_displayName' : athlete.get('athlete',{}).get('displayName')
                }

                for index, stat in enumerate(athlete.get('stats',[])):
                    if '-' not in player_dict_keys[index]:
                        athlete_boxScore_dict[f"{player_dict_keys[index]}"] = stat
                    else:
                        key1 = player_dict_keys[index].split('-')[0]
                        key2 = player_dict_keys[index].split('-')[1]

                        value1 = stat.split('-')[0]
                        value2 = stat.split('-')[1]
                        athlete_boxScore_dict[key1] = value1
                        athlete_boxScore_dict[key2] = value2
                

                playerBoxScores_df.append(athlete_boxScore_dict)
        predictor_df = []
        base_predictor_dict = event_data.get('predictor',{})
        predictor_dict = {'event_id' : event_id}
        for team in ['awayTeam', 'homeTeam']:
            team_predictor_dict = base_predictor_dict.get(team,{})
            predictor_dict[f"{team}_id"] = team_predictor_dict.get('id')
            if 'gameProjection' in team_predictor_dict:
                predictor_dict[f"{team}_gameProjection"] = team_predictor_dict['gameProjection']
            else:
                otherTeam = [ot for ot in ['awayTeam', 'homeTeam'] if ot != team][0]
                predictor_dict[f"{team}_gameProjection"] = base_predictor_dict.get(otherTeam,{}).get('teamChanceLoss')
        predictor_df.append(predictor_dict)


        play_by_play_df = []

        for play in event_data.get('plays',[]):

            play_by_play_dict = {
                'game_id' : event_id,
                'play_id' : play.get('id'),
                'sequenceNumber' : play.get('sequenceNumber'),
                'play_type_id' : play.get('type',{}).get('id'),
                'play_type_text' : play.get('type',{}).get('text'),
                'play_text' : play.get('text'),
                'awayScore' : play.get('awayScore'),
                'homeScore' : play.get('homeScore'),
                'period_number' : play.get('period',{}).get('number'),
                'period_displayValue' : play.get('period',{}).get('displayValue'),
                'clock_displayValue' : play.get('clock',{}).get('displayValue'),
                'isScoringPlay' : play.get('scoringPlay'),
                'scoreValue' : play.get('scoreValue'),
                'offenseTeam' : play.get('team',{}).get('id'),
                'isShootingPlay' : play.get('shootingPlay'),
                'x_coordinate' : play.get('coordinate',{}).get('x'),
                'y_coordinate' : play.get('coordinate',{}).get('y')

            }
            play_by_play_df.append(play_by_play_dict)


        return teamBoxScores_df, playerBoxScores_df, predictor_df, play_by_play_df

        

    except Exception as e:
        print(f"Error fetching data for URL {event_url}: {e}")
        return [], [], [], []





In [8]:
teamBoxScores_all, playerBoxScores_all, predictor_all, play_by_play_all = [], [], [], []

with ThreadPoolExecutor(max_workers=100) as executor:  # Adjust max_workers as needed
    future_to_event = {executor.submit(grab_event_data, event_id): event_id for event_id in new_ids}

    for future in as_completed(future_to_event):
        event_id = future_to_event[future]
        try:
            teamBoxScores, playerBoxScores, predictor, play_by_play = future.result()
            teamBoxScores_all.extend(teamBoxScores)
            playerBoxScores_all.extend(playerBoxScores)
            predictor_all.extend(predictor)
            play_by_play_all.extend(play_by_play)
        except Exception as e:
            print(f"Error processing event {event_id}: {e}")

# Convert results to DataFrames
# teamBoxScores_df = pd.DataFrame(teamBoxScores_all)
# playerBoxScores_df = pd.DataFrame(playerBoxScores_all)
# predictor_df = pd.DataFrame(predictor_all)
# play_by_play_df = pd.DataFrame(play_by_play_all)

In [9]:
conn = sqlite3.connect("nbaDatasets/nba_database.db")
cursor = conn.cursor()

# Loop through the DataFrame rows and insert data
for index, row in sched_df.iterrows():
    # Convert `event_startTime` to string (if it's not already a string)
    start_time = row['event_startTime']
    if not isinstance(start_time, str):
        start_time = start_time.strftime("%H:%M:%S")  # Assuming it's a `datetime.time` object

    cursor.execute("""
    INSERT OR REPLACE INTO Schedule (
        event_id,
        event_season,
        event_neutralSite,
        event_boxScoreAvailable,
        venue_name,
        home_id,
        home_score,
        away_id,
        away_score,
        status_completed,
        status_description,
        event_startDate,
        event_startTime
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['event_id'],
        row['event_season'],
        row['event_neutralSite'],
        row['event_boxScoreAvailable'],
        row['venue_name'],
        row['home_id'],
        row['home_score'],
        row['away_id'],
        row['away_score'],
        row['status_completed'],
        row['status_description'],
        row['event_startDate'],
        start_time  # Insert the time as a string
    ))

# Commit changes and close the connection
conn.commit()
conn.close()


/var/folders/_2/5b73f0k12nvbrc5znk3nk17h0000gr/T/ipykernel_37894/2269472066.py:11: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


In [10]:
conn = sqlite3.connect("nbaDatasets/nba_database.db")
cursor = conn.cursor()


# Create and Add Player Boxscores
cursor.execute("""
CREATE TABLE IF NOT EXISTS playerBoxScores (
    event_id TEXT,
    team_id INTEGER,
    team_displayName TEXT,
    athlete_active BOOLEAN,
    athlete_starter BOOLEAN,
    athlete_didNotPlay BOOLEAN,
    athlete_reason TEXT,
    athlete_ejected BOOLEAN,
    athlete_id INTEGER,
    athlete_displayName TEXT,
    minutes INTEGER,
    fieldGoalsMade INTEGER,
    fieldGoalsAttempted INTEGER,
    threePointFieldGoalsMade INTEGER,
    threePointFieldGoalsAttempted INTEGER,
    freeThrowsMade INTEGER,
    freeThrowsAttempted INTEGER,
    offensiveRebounds INTEGER,
    defensiveRebounds INTEGER,
    rebounds INTEGER,
    assists INTEGER,
    steals INTEGER,
    blocks INTEGER,
    turnovers INTEGER,
    fouls INTEGER,
    plusMinus INTEGER,
    points INTEGER,
    PRIMARY KEY (event_id, athlete_id)
);
""")

# # Loop through the DataFrame rows and insert data
for index, row in enumerate(playerBoxScores_all):

    cursor.execute("""
    INSERT OR IGNORE INTO playerBoxScores (
    event_id,
    team_id,
    team_displayName,
    athlete_active,
    athlete_starter,
    athlete_didNotPlay,
    athlete_reason,
    athlete_ejected,
    athlete_id,
    athlete_displayName,
    minutes,
    fieldGoalsMade,
    fieldGoalsAttempted,
    threePointFieldGoalsMade,
    threePointFieldGoalsAttempted,
    freeThrowsMade,
    freeThrowsAttempted,
    offensiveRebounds,
    defensiveRebounds,
    rebounds,
    assists,
    steals,
    blocks,
    turnovers,
    fouls,
    plusMinus,
    points
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? , ? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,?)
    """, (
        row['event_id'],
        row['team_id'],
        row['team_displayName'],
        row['athlete_active'],
        row['athlete_starter'],
        row['athlete_didNotPlay'],
        row['athlete_reason'],
        row['athlete_ejected'],
        row['athlete_id'],
        row['athlete_displayName'],
        row.get('minutes'),
        row.get('fieldGoalsMade'),
        row.get('fieldGoalsAttempted'),
        row.get('threePointFieldGoalsMade'),
        row.get('threePointFieldGoalsAttempted'),
        row.get('freeThrowsMade'),
        row.get('freeThrowsAttempted'),
        row.get('offensiveRebounds'),
        row.get('defensiveRebounds'),
        row.get('rebounds'),
        row.get('assists'),
        row.get('steals'),
        row.get('blocks'),
        row.get('turnovers'),
        row.get('fouls'),
        row.get('plusMinus'),
        row.get('points'),
    ))

# Create and Add Team Boxscores
cursor.execute("""
CREATE TABLE IF NOT EXISTS teamBoxScores (
    event_id INTEGER,
    team_id INTEGER,
    fieldGoalsMade INTEGER,
    fieldGoalsAttempted INTEGER,
    fieldGoalPct REAL,
    threePointFieldGoalsMade INTEGER,
    threePointFieldGoalsAttempted INTEGER,
    threePointFieldGoalPct REAL,
    freeThrowsMade INTEGER,
    freeThrowsAttempted INTEGER,
    freeThrowPct REAL,
    totalRebounds INTEGER,
    offensiveRebounds INTEGER,
    defensiveRebounds INTEGER,
    assists INTEGER,
    steals INTEGER,
    blocks INTEGER,
    turnovers INTEGER,
    teamTurnovers INTEGER,
    totalTurnovers INTEGER,
    technicalFouls INTEGER,
    totalTechnicalFouls INTEGER,
    flagrantFouls INTEGER,
    turnoverPoints INTEGER,
    fastBreakPoints INTEGER,
    pointsInPaint INTEGER,
    fouls INTEGER,
    largestLead INTEGER,
    PRIMARY KEY (event_id, team_id)
);
""")

for row in teamBoxScores_all:
    cursor.execute("""
    INSERT OR IGNORE INTO teamBoxScores (
        event_id,
        team_id,
        fieldGoalsMade,
        fieldGoalsAttempted,
        fieldGoalPct,
        threePointFieldGoalsMade,
        threePointFieldGoalsAttempted,
        threePointFieldGoalPct,
        freeThrowsMade,
        freeThrowsAttempted,
        freeThrowPct,
        totalRebounds,
        offensiveRebounds,
        defensiveRebounds,
        assists,
        steals,
        blocks,
        turnovers,
        teamTurnovers,
        totalTurnovers,
        technicalFouls,
        totalTechnicalFouls,
        flagrantFouls,
        turnoverPoints,
        fastBreakPoints,
        pointsInPaint,
        fouls,
        largestLead
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)

    """, (
        row['event_id'],
        row['team_id'],
        row.get('fieldGoalsMade'),
        row.get('fieldGoalsAttempted'),
        row.get('fieldGoalPct'),
        row.get('threePointFieldGoalsMade'),
        row.get('threePointFieldGoalsAttempted'),
        row.get('threePointFieldGoalPct'),
        row.get('freeThrowsMade'),
        row.get('freeThrowsAttempted'),
        row.get('freeThrowPct'),
        row.get('totalRebounds'),
        row.get('offensiveRebounds'),
        row.get('defensiveRebounds'),
        row.get('assists'),
        row.get('steals'),
        row.get('blocks'),
        row.get('turnovers'),
        row.get('teamTurnovers'),
        row.get('totalTurnovers'),
        row.get('technicalFouls'),
        row.get('totalTechnicalFouls'),
        row.get('flagrantFouls'),
        row.get('turnoverPoints'),
        row.get('fastBreakPoints'),
        row.get('pointsInPaint'),
        row.get('fouls'),
        row.get('largestLead')
    ))


# Create and Add Prediction
cursor.execute("""
CREATE TABLE IF NOT EXISTS Predictions (
    event_id INTEGER,
    awayTeam_id INTEGER,
    awayTeam_gameProjection REAL,
    homeTeam_id INTEGER,
    homeTeam_gameProjection REAL,
    PRIMARY KEY (event_id, awayTeam_id, homeTeam_id)
)
               """)


cursor.executemany("""
INSERT OR IGNORE INTO Predictions (
    event_id,
    awayTeam_id,
    awayTeam_gameProjection,
    homeTeam_id,
    homeTeam_gameProjection
) VALUES (?, ?, ?, ?, ?)
""", [
    (
        pred["event_id"],
        pred["awayTeam_id"],
        pred["awayTeam_gameProjection"],
        pred["homeTeam_id"],
        pred["homeTeam_gameProjection"]
    )
    for pred in predictor_all
])

# Create and Add Play by Play

cursor.execute("""
CREATE TABLE IF NOT EXISTS playByPlay (
    game_id INTEGER,
    play_id INTEGER,
    sequenceNumber INTEGER,
    play_type_id INTEGER,
    play_type_text TEXT,
    play_text TEXT,
    awayScore INTEGER,
    homeScore INTEGER,
    period_number INTEGER,
    period_displayValue TEXT,
    clock_displayValue TEXT,
    isScoringPlay BOOLEAN,
    scoreValue INTEGER,
    offenseTeam INTEGER,
    isShootingPlay BOOLEAN,
    x_coordinate REAL,
    y_coordinate REAL,
    PRIMARY KEY (game_id, play_id)
);
""")

cursor.executemany("""
INSERT OR IGNORE INTO playByPlay (
    game_id,
    play_id,
    sequenceNumber,
    play_type_id,
    play_type_text,
    play_text,
    awayScore,
    homeScore,
    period_number,
    period_displayValue,
    clock_displayValue,
    isScoringPlay,
    scoreValue,
    offenseTeam,
    isShootingPlay,
    x_coordinate,
    y_coordinate
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", [
    (
        play["game_id"],
        play["play_id"],
        play["sequenceNumber"],
        play["play_type_id"],
        play["play_type_text"],
        play["play_text"],
        play["awayScore"],
        play["homeScore"],
        play["period_number"],
        play["period_displayValue"],
        play["clock_displayValue"],
        play["isScoringPlay"],
        play["scoreValue"],
        play["offenseTeam"],
        play["isShootingPlay"],
        play["x_coordinate"],
        play["y_coordinate"]
    )
    for play in play_by_play_all
])


conn.commit()
conn.close()